In [1]:
import os.path as path
import raw_data_preprocessing as rdp

googledrive_loc = '/datasets/googledrive'
data_folder_loc = 'Yi_UCI_research/GSR other works/2020 Summer_predict individual training/data/'
file_name = 'raw data/yi feng larger dataset.csv'
data_path = path.join(googledrive_loc, data_folder_loc, file_name)
raw_data = rdp.read_raw_data(data_path)
print('Used Feature columns: {}'.format(rdp.feature_col_names))
data_no_missing = rdp.delete_missing_row(raw_data)


Used Feature columns: ['Sleep', 'SES', 'VGQHoursSumPastYear', 'CFQ', 'TCA', 'Workmastery', 'Grit', 'Ambition', 'PSS', 'Competitiveness', 'MRpre', 'IC_pre', 'WM_pre', 'Updating_acc_zscore', 'Person_extraver', 'Person_agreeable', 'Person_conscien', 'Person_emot', 'Person_opennes']
The sample size of raw data is 281.
The sample size of no missing data is 262.


In [2]:
import intervention_data_preprocessing as idp

x = range(1,10)
y = [1,1,1,1,1,1,1,1,1]
# result = log_reg.fit_row(x,y)
piece_lin_reg = idp.PiecewiselinRegressor(['knot', 'slope1', 'slope2'])
result = piece_lin_reg.fit_row(x, y)
print(result)
test = [1,1,1,1,1,1]
print(max(test))

{'parameters': [nan, nan, nan], 'r2': nan}
1


In [3]:
# piece_em_processor = raw_data_preprocessing.InterventionProcessor(intervention_data)
# piece_em_processor.register_regressor(raw_data_preprocessing.LogRegressor(0))
# piece_em_processor.fit()

In [4]:
intervention_data = idp.get_intervention_data(data_no_missing)
piece_lin_reg.fit(intervention_data)
intervention_data

/work/IndividualDifference/intervention_data_preprocessing.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[pn] = np.nan


,WmeanN_1,WmeanN_2,WmeanN_3,WmeanN_4,WmeanN_5,WmeanN_6,WmeanN_7,WmeanN_8,WmeanN_9,WmeanN_10,knot,slope1,slope2
0,2.532189,2.951220,3.981067,3.069697,6.489933,8.599343,3.490166,5.315956,6.595960,6.297114,6.0,0.828884,-0.132875
1,2.004138,2.490342,2.164074,2.579515,4.816541,4.362275,4.104225,2.912553,4.726727,4.812147,5.0,0.544715,0.112200
2,3.025449,3.758287,3.861799,3.580940,4.017241,5.250646,4.645780,5.345315,4.882199,4.956466,8.0,0.294313,-0.164472
3,2.552137,2.758261,3.335548,3.615517,3.603839,4.394864,4.600334,4.544872,5.965574,5.740260,8.0,0.335222,0.560211
4,1.738197,2.528198,2.299090,2.566576,3.322917,3.460189,3.578313,3.400264,3.597671,2.701987,7.0,0.309710,-0.261232
...,...,...,...,...,...,...,...,...,...,...,...,...,...
275,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,NaN,NaN,NaN
276,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,NaN,NaN,NaN
277,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,NaN,NaN,NaN
278,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,NaN,NaN,NaN


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=6b8ead81-70bf-4d6b-bd50-8194f8f72b46' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>